# Mixture of logistic regression models for cell-type specific methylation

### Model of DNA methylation for a purified/isolated cell type:

Let $p_n$ be the probability of having methylation at position $n$ in the genome; we assume $N$ sites that are eligible for methylation, $n = 1 ... N$. Let $x_{n,m}$ be the observed methylation at position $n$ on read $m$

$$x_{n,m} = \left\{\begin{matrix} 1~,\hspace{22px}\text{if methylated} \\ 0~,~\text{not methylated}  \end{matrix}\right.$$

Each read is much shorter than the entire genome, so we have to figure out how to deal with missing data. 

$$x_{n,m} = \left\{\begin{matrix} 1~,\hspace{40px}\text{if methylated}\hspace{40px} \\ 0~,\hspace{35px}\text{not methylated}\hspace{35px} \\ *~,~\text{position $n$ not in read $m$} \end{matrix}\right.$$

I'm not 100% sure the best way to deal with missing data yet (see "concerns" below). In principle, the next step would be to do a logistic regression, using the observed methylations to the left/right of position $i$ to predict the probability of methylation:

$$ \ln \left( \frac{\hat{p}_i}{1 - \hat{p}_i} \right) = \beta_0 + \beta_1 x_{i+1} + \beta_{-1} x_{i-1} + \beta_2 x_{i+2} + \beta_{-2} x_{i-2} + ...$$

The above equation models our estimate of $p_i$ given a single read that contains a $0$ or $1$ for $x_{i,m}$ (we omit the $m$ since it is the same for all variables). More compactly,

$$ \ln \left( \frac{\hat{p}_i}{1 - \hat{p}_i} \right) = \beta_0 + \boldsymbol{\beta}_+ \mathbf{x}_{i+} + \boldsymbol{\beta}_- \mathbf{x}_{i-}$$

Where $\boldsymbol{\beta}_+ = [\beta_1, \beta_2, ..., \beta_k]^T$ are parameters for $\mathbf{x}_{i+} = [x_{i+1}, x_{i+2}, ..., x_{i+k}]^T$ and $\boldsymbol{\beta}_- = [\beta_{-1}, \beta_{-2}, ..., \beta_{-k}]^T$ are parameters for $\mathbf{x}_{i-} = [x_{i-1}, x_{i-2}, ..., x_{i-k}]^T$. This can be thought of as an autoregressive model of order $k$ in both directions from methylation site $i$.


### Extension to multiple cell types or clusters

If we have a mixed sample with multiple cell types (clusters), then we can model the probability of methylation at each site with a mixture of the logistic models described above. Suppose we have $C$ different cell types:

$$ \ln \left( \frac{\hat{p}_i}{1 - \hat{p}_i} \right) = \left\{\begin{matrix}
\beta_0^1 + \boldsymbol{\beta}_+^1 \mathbf{x}_{i+} + \boldsymbol{\beta}_{-}^1 \mathbf{x}_{i-}
\hspace{20px},\hspace{8px}\text{with probability } \pi_1 \\
\beta_0^2 + \boldsymbol{\beta}_+^2 \mathbf{x}_{i+} + \boldsymbol{\beta}_{-}^2 \mathbf{x}_{i-}
\hspace{20px},\hspace{8px}\text{with probability } \pi_2 \\
... \\
\beta_0^C + \boldsymbol{\beta}_+^C \mathbf{x}_{i+} + \boldsymbol{\beta}_{-}^C \mathbf{x}_{i-}
\hspace{20px},\hspace{8px}\text{with probability } \pi_C \end{matrix}\right.$$

With $\boldsymbol{\pi} = [\pi_1, \pi_2, ..., \pi_C]^T$ being the proportions of each cell type in the sample, $\sum_c \pi_c = 1$.

[Fariaa & Soromenhob (2010)](http://www.researchgate.net/profile/S_Faria/publication/228464588_Fitting_mixtures_of_linear_regressions/links/00b49519b60d62e512000000.pdf) describe and compare methods for fitting mixtures of linear regression models. There is a library called [FlexMix](http://cran.r-project.org/web/packages/flexmix/vignettes/flexmix-intro.pdf) in R that implements some of these algorithms. Also see [these lecture slides](https://stat.duke.edu/courses/Fall08/sta216/lecture13.pdf).

### Concerns

* Clusters are of different sizes (some cell types much more common than others).
    * http://www-users.cs.umn.edu/~kumar/papers/kdd02_snn_28.pdf
* How to deal with the missing data
    * The [default behavior for R](http://stats.stackexchange.com/questions/11000/how-does-r-handle-missing-values-in-lm) is to omit any observation that does not contain complete information. Thus, for a large $k$, we would throw away most reads, and lose a lot of information.
    * Maybe try data imputation? When position $n$ is not available for read $m$, then set $x_{n,m}$ to the observed methylation probability at position $n$?
* Computational tractability
    * If we fit a different mixture of models to predict the probability of methylation at each eligible position, this could take a lot of compute time. However, it is embarassingly parallel, so access to a cluster *might* make this plausible.